In [29]:
import numpy as np
import theano
from theano import tensor as T
import lasagne
from lasagne.layers import *
from lasagne.nonlinearities import *

In [18]:
class UpperRightOnesLayer(Layer):
    def __init__(self, incoming, **kwargs):
        super(UpperRightOnesLayer, self).__init__(incoming, **kwargs)
        num_inputs = int(np.prod(self.input_shape[1:]))
        self.num_inputs = num_inputs
        # construct the upper triangular matrix
        self.W = np.ones((num_inputs, num_inputs), dtype="float32")
        for k in range(0, num_inputs):
            self.W[k][0:k] = 0

    def get_output_shape_for(self, input_shape):
        return (input_shape[0], self.num_inputs)

    def get_output_for(self, input, **kwargs):
        if input.ndim > 2:
            # if the input has more than two dimensions, flatten it into a
            # batch of feature vectors.
            input = input.flatten(2)

        result = T.dot(input, self.W)
        return result

In [14]:
W = np.ones((3, 3), dtype="float32")
for k in range(0, 3):
    W[k][0:k] = 0

In [15]:
W

array([[ 1.,  1.,  1.],
       [ 0.,  1.,  1.],
       [ 0.,  0.,  1.]], dtype=float32)

In [30]:
l_in = InputLayer((None, 3))
l_uro = UpperRightOnesLayer(l_in)
l_softmax = NonlinearityLayer(l_uro, nonlinearity=softmax)

In [31]:
X = T.fmatrix('X')
uro_out = get_output(l_uro, X)
softmax_out = get_output(l_softmax, X)

In [28]:
uro_out.eval({X: np.zeros((3,3), dtype="float32")+0.5})

array([[ 0.5,  1. ,  1.5],
       [ 0.5,  1. ,  1.5],
       [ 0.5,  1. ,  1.5]], dtype=float32)

In [32]:
softmax_out.eval({X: np.zeros((3,3), dtype="float32")+0.5})

array([[ 0.18632373,  0.30719587,  0.5064804 ],
       [ 0.18632373,  0.30719587,  0.5064804 ],
       [ 0.18632373,  0.30719587,  0.5064804 ]], dtype=float32)

---

In [54]:
class OrdinalSubtractLayer(Layer):
    def __init__(self, incoming, **kwargs):
        super(OrdinalSubtractLayer, self).__init__(incoming, **kwargs)
        num_inputs = int(np.prod(self.input_shape[1:]))
        self.num_inputs = num_inputs
        # construct the matrix
        self.W = np.zeros((num_inputs, num_inputs), dtype="float32")
        self.W[0,0]=1
        for k in range(1, num_inputs-1):
            self.W[k-1,k] = -1
            self.W[k,k] = 1
        self.W[num_inputs-1,num_inputs-1] = 1
        # construct the bias row vector
        self.b = np.zeros((1, num_inputs), dtype="float32")
        self.b[0, num_inputs-1] = 1

    def get_output_shape_for(self, input_shape):
        return (input_shape[0], self.num_inputs)

    def get_output_for(self, input, **kwargs):
        if input.ndim > 2:
            # if the input has more than two dimensions, flatten it into a
            # batch of feature vectors.
            input = input.flatten(2)

        result = T.dot(input, self.W)
        result = T.abs_(self.b - result)
        return result

In [55]:
l_ord = OrdinalSubtractLayer(l_softmax)
ord_out = get_output(l_ord, X)

In [56]:
ord_out.eval({X: np.zeros((3,3), dtype="float32")+0.5})

array([[ 0.18632373,  0.12087214,  0.4935196 ],
       [ 0.18632373,  0.12087214,  0.4935196 ],
       [ 0.18632373,  0.12087214,  0.4935196 ]], dtype=float32)

In [42]:
num_inputs=3
W = np.zeros((num_inputs, num_inputs))
W[0,0]=1
for k in range(1, num_inputs-1):
    W[k-1,k] = -1
    W[k,k] = 1
W[num_inputs-1,num_inputs-1] = 1
W

array([[ 1., -1.,  0.],
       [ 0.,  1.,  0.],
       [ 0.,  0.,  1.]])

In [48]:
(T.ones((1,4)) - (T.ones((10,4))*2)).eval()

array([[-1., -1., -1., -1.],
       [-1., -1., -1., -1.],
       [-1., -1., -1., -1.],
       [-1., -1., -1., -1.],
       [-1., -1., -1., -1.],
       [-1., -1., -1., -1.],
       [-1., -1., -1., -1.],
       [-1., -1., -1., -1.],
       [-1., -1., -1., -1.],
       [-1., -1., -1., -1.]])